# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [4]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))



# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [6]:
from sklearn.model_selection import train_test_split  # Import train_test_split

X = adult_dt.drop(columns='income')  # Features DataFrame
Y = adult_dt['income']  # Target DataFrame

# Split data into training and testing sets with 70-30% split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Display shapes of the resulting splits
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((22792, 14), (9769, 14), (22792,), (9769,))

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*
The random state in the train_test_split function is a parameter that controls the randomness of the data split. It serves as a "seed" for the random number generator, ensuring that the split is reproducible.

It Useful because 
Reproducibility:
By setting a fixed random_state (e.g., random_state=42), we ensure that every time we run the code, we get the same train-test split. This consistency allows us to reproduce our experiments exactly and compare results reliably.
Consistent Evaluation:
In machine learning, using a consistent dataset split allows us to fairly evaluate different models or configurations. By keeping the train and test data the same across experiments, we ensure that any differences in model performance are due to the models themselves, not variations in the data split.
Collaboration:
A fixed random state is useful for collaborative work. When multiple people or teams run the same code with the same random state, they will get identical results, which facilitates shared analyses and debugging.


# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [11]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import set_config

# Define numerical and categorical columns
numerical_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Define the transformations for numerical columns
num_transforms = Pipeline(steps=[
    ('KNNImputer', KNNImputer(n_neighbors=7, weights='distance')),  # KNN Imputer with 7 nearest neighbors
    ('RobustScaler', RobustScaler())  # RobustScaler for outlier-resistant scaling
])

# Define the transformations for categorical columns
cat_transforms = Pipeline(steps=[
    ('SimpleImputer', SimpleImputer(strategy='most_frequent')),  # Simple Imputer using mode
    ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))  # One-hot encoding
])

# Combine transformations in ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', num_transforms, numerical_cols),
        ('cat_transforms', cat_transforms, categorical_cols)
    ]
)
set_config(display='diagram')


# Test the ColumnTransformer on the dataset
preprocessed_data = preprocessor.fit_transform(adult_dt)
print(preprocessed_data.shape)

preprocessor

(32561, 107)


ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('KNNImputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('RobustScaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('SimpleImputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Create the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # Add ColumnTransformer from previous section
    ('classifier', RandomForestClassifier(random_state=42))  # Add RandomForestClassifier
])

# Display the pipeline to verify
model_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('KNNImputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('SimpleImputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OneHotEncoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [13]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Define the scoring metrics
scoring = {
    'neg_log_loss': make_scorer(log_loss, needs_proba=True),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}

# Evaluate the model pipeline using cross-validation
cv_results = cross_validate(model_pipeline, adult_dt, Y, cv=5, scoring=scoring, return_train_score=True)

# Report training and validation results
print("Training and Validation Results:")
print(f"Training Negative Log Loss: {-cv_results['train_neg_log_loss'].mean():.4f}")
print(f"Validation Negative Log Loss: {-cv_results['test_neg_log_loss'].mean():.4f}")
print(f"Training ROC AUC: {cv_results['train_roc_auc'].mean():.4f}")
print(f"Validation ROC AUC: {cv_results['test_roc_auc'].mean():.4f}")
print(f"Training Accuracy: {cv_results['train_accuracy'].mean():.4f}")
print(f"Validation Accuracy: {cv_results['test_accuracy'].mean():.4f}")
print(f"Training Balanced Accuracy: {cv_results['train_balanced_accuracy'].mean():.4f}")
print(f"Validation Balanced Accuracy: {cv_results['test_balanced_accuracy'].mean():.4f}")


d:\Users\Besher\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
d:\Users\Besher\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
d:\Users\Besher\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training and Validation Results:
Training Negative Log Loss: -0.0806
Validation Negative Log Loss: -0.3751
Training ROC AUC: 1.0000
Validation ROC AUC: 0.7754
Training Accuracy: 1.0000
Validation Accuracy: 0.8547
Training Balanced Accuracy: 1.0000
Validation Balanced Accuracy: 0.7754


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [14]:
import pandas as pd

# Extract fold-level results
results_df = pd.DataFrame({
    'fold': range(1, 6),  # Assuming 5 folds
    'train_neg_log_loss': cv_results['train_neg_log_loss'],
    'test_neg_log_loss': cv_results['test_neg_log_loss'],
    'train_roc_auc': cv_results['train_roc_auc'],
    'test_roc_auc': cv_results['test_roc_auc'],
    'train_accuracy': cv_results['train_accuracy'],
    'test_accuracy': cv_results['test_accuracy'],
    'train_balanced_accuracy': cv_results['train_balanced_accuracy'],
    'test_balanced_accuracy': cv_results['test_balanced_accuracy']
})

# Sort the DataFrame by test negative log loss
sorted_results_df = results_df.sort_values(by='test_neg_log_loss', ascending=True)

# Display the sorted results
sorted_results_df.reset_index(drop=True, inplace=True)  # Reset index for better presentation
print(sorted_results_df)

   fold  train_neg_log_loss  test_neg_log_loss  train_roc_auc  test_roc_auc  \
0     4            0.080591           0.356018       1.000000      0.776560   
1     3            0.080695           0.369435       0.999975      0.778277   
2     5            0.081341           0.375717       0.999975      0.782353   
3     1            0.080468           0.379042       0.999920      0.770268   
4     2            0.079882           0.395460       0.999975      0.769466   

   train_accuracy  test_accuracy  train_balanced_accuracy  \
0        1.000000       0.858108                 1.000000   
1        0.999962       0.852119                 0.999975   
2        0.999962       0.857647                 0.999975   
3        0.999962       0.853677                 0.999920   
4        0.999962       0.851966                 0.999975   

   test_balanced_accuracy  
0                0.776560  
1                0.778277  
2                0.782353  
3                0.770268  
4                0

Calculate the mean of each metric. 

In [15]:
# Calculate the mean of each metric
mean_metrics = {
    'mean_train_neg_log_loss': -results_df['train_neg_log_loss'].mean(),
    'mean_test_neg_log_loss': -results_df['test_neg_log_loss'].mean(),
    'mean_train_roc_auc': results_df['train_roc_auc'].mean(),
    'mean_test_roc_auc': results_df['test_roc_auc'].mean(),
    'mean_train_accuracy': results_df['train_accuracy'].mean(),
    'mean_test_accuracy': results_df['test_accuracy'].mean(),
    'mean_train_balanced_accuracy': results_df['train_balanced_accuracy'].mean(),
    'mean_test_balanced_accuracy': results_df['test_balanced_accuracy'].mean()
}

# Convert the results to a DataFrame for better visualization
mean_metrics_df = pd.DataFrame(mean_metrics, index=[0])

# Display the mean metrics
print(mean_metrics_df)


   mean_train_neg_log_loss  mean_test_neg_log_loss  mean_train_roc_auc  \
0                -0.080596               -0.375134            0.999969   

   mean_test_roc_auc  mean_train_accuracy  mean_test_accuracy  \
0           0.775385             0.999969            0.854704   

   mean_train_balanced_accuracy  mean_test_balanced_accuracy  
0                      0.999969                     0.775385  


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.